In [ ]:
from azureml.core import Workspace, Experiment

ws = Workspace.get(name="udacity-project")
exp = Experiment(workspace=ws, name="udacity-project")

print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')

run = exp.start_logging()

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute

# TODO: Create compute cluster
# Use vm_size = "Standard_D2_V2" in your provisioning configuration.
# max_nodes should be no greater than 4.

compute_configurations = AmlCompute.provisioning_configuration(vm_size='STANDARD_D2_V2', max_nodes=4)
compute_cluster = ComputeTarget.create(ws, 'compute_cluster', compute_configurations)
compute_cluster.wait_for_completion(show_output=True)

In [ ]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import uniform, choice
import os

# Specify parameter sampler
ps = RandomParameterSampling({
    '--C': uniform(0.05, 0.1),
    '--max_iter': choice(0, 50, 100, 150),
})

# Specify a Policy
policy = BanditPolicy(slack_factor = 0.1, evaluation_interval=2)

if "training" not in os.listdir():
    os.mkdir("./training")

# Create a SKLearn estimator for use with train.py
est = SKLearn(source_directory='./',
              script_params={
                  "--C": 1.0,
                  "--max_iter": 100
              },
              entry_script='train.py',
              compute_target=compute_cluster)

# Create a HyperDriveConfig using the estimator, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(hyperparameter_sampling=ps,
                                     policy=policy,
                                     primary_metric_name='Accuracy',
                                     primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                     max_total_runs=12,
                                     max_concurrent_runs=4,
                                     estimator=est)

In [ ]:
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyper_run = exp.submit(config=hyperdrive_config, show_output=True)

In [ ]:
import joblib
# Get your best run and save the model from that run.

best_run = hyper_run.get_best_run_by_primary_metric()
best_run_metrics = best_run.get_metrics()
parameter_values = best_run.get_details()['runDefinition']['Arguments']

joblib.dump(value=best_hyperdrive_run.get_details()['runDefinition']['arguments'], filename=os.path.join('training', 'best_hyperdrive_model.joblib'))
print(f'accurracy: {best_run_metrics['Accuracy']}')

In [ ]:
from azureml.data.dataset_factory import TabularDatasetFactory

# Create TabularDataset using TabularDatasetFactory
# Data is available at: 
# "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

path = "https://automlsamplenotebookdata.blob.core.windows.net/automl-sample-notebook-data/bankmarketing_train.csv"

data_store = TabularDatasetFactory.from_delimited_files(path, validate=True, include_path=False, infer_column_types=True, set_column_types=None, separator=',', header=True, partition_format=None, support_multi_line=False, empty_as_string=False, encoding='utf8')

In [ ]:
from train import clean_data

# Use the clean_data function to clean your data.
x, y = clean_data(data_store)
training_data = pd.concat([x, y], axis=1, join='inner')

In [ ]:
from azureml.train.automl import AutoMLConfig

# Set parameters for AutoMLConfig
# NOTE: DO NOT CHANGE THE experiment_timeout_minutes PARAMETER OR YOUR INSTANCE WILL TIME OUT.
# If you wish to run the experiment longer, you will need to run this notebook in your own
# Azure tenant, which will incur personal costs.
automl_config = AutoMLConfig(
    experiment_timeout_minutes=30,
    task='classification',
    debug_log='automl_debug.log',
    comput_target=compute_cluster,
    primary_metric='Accuracy',
    training_data=training_data,
    label_column_name='y',
    enable_early_stopping=True,
    n_cross_validations=5)

In [2]:
# Submit your automl run

auto_run = exp.submit(config=automl_config, show_output=True)

In [ ]:
# Retrieve and save your best automl model.
best_auto_run = auto_run.get_best_run_by_primary_metric()
best_auto_run_metrics = best_auto_run.get_metrics()
auto_parameter_values = best_auto_run.get_details()

joblib.dump(value=auto_parameter_values, filename='./training/autorun_model.joblib')
print(f'accurracy: {best_auto_run_metricssq['Accuracy']}')